FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\data_object_velodyne/training/velodyne/000000.bin'

In [5]:
from datasets import load_dataset

d:\miniconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

ds = load_dataset("itsanmolgupta/mimic-cxr-dataset", cache_dir="./nithish")

Generating train split: 100%|██████████| 30633/30633 [00:01<00:00, 28574.97 examples/s]


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Load model & tokenizer
model_name = "gpt2"  # replace with your LLM
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Small dataset (toy text)
texts = ["Hello world", "Backpropagation teaches networks", "Loss functions guide learning"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# 3. Define loss function (CrossEntropyLoss)
loss_fn = torch.nn.CrossEntropyLoss()

# 4. Flatten parameters into a vector
params = torch.cat([p.data.view(-1) for p in model.parameters()])
dim = params.numel()

# 5. Create random directions (normalized)
d1 = torch.randn(dim)
d2 = torch.randn(dim)
d1 = d1 / torch.norm(d1)
d2 = d2 / torch.norm(d2)

# 6. Function to set new weights and compute loss
def compute_loss(offset1, offset2):
    new_params = params + offset1 * d1 + offset2 * d2
    pointer = 0
    for p in model.parameters():
        num = p.numel()
        p.data = new_params[pointer:pointer+num].view_as(p).clone()
        pointer += num

    outputs = model(**inputs, labels=inputs["input_ids"])
    return outputs.loss.item()

# 7. Grid of offsets
alphas = np.linspace(-1, 1, 30)
betas = np.linspace(-1, 1, 30)
loss_surface = np.zeros((len(alphas), len(betas)))

for i, a in enumerate(alphas):
    for j, b in enumerate(betas):
        loss_surface[j, i] = compute_loss(a, b)

# 8. Plot
plt.figure(figsize=(8,6))
plt.contourf(alphas, betas, loss_surface, levels=50, cmap='viridis')
plt.colorbar(label="Loss")
plt.xlabel("Direction 1 (α)")
plt.ylabel("Direction 2 (β)")
plt.title("Loss Landscape of LLM (2D projection)")
plt.show()


d:\miniconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.